# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hermanus,-34.4187,19.2345,17.74,86,5,4.76,ZA,1701899019
1,1,san pedro pochutla,15.7333,-96.4667,30.84,61,80,3.01,MX,1701899019
2,2,bilibino,68.0546,166.4372,-30.91,99,100,1.83,RU,1701899019
3,3,la crosse,43.8014,-91.2396,7.42,60,0,7.20,US,1701898857
4,4,badger,64.8000,-147.5333,-14.07,84,100,1.54,US,1701899020


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM", size = "Humidity", color ='purple')
# Display the map
humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = pd.DataFrame(city_data_df.loc[(city_data_df['Max Temp']>21) & 
                                              (city_data_df['Max Temp']<27) & 
                                              (city_data_df['Humidity']<60) &
                                              (city_data_df['Wind Speed']<4.5)])

# Drop any rows with null values
ideal_city_df.dropna()

# Display sample data
ideal_city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
19,19,taoudenni,22.6783,-3.9836,22.55,11,100,2.87,ML,1701899022
80,80,sur,22.5667,59.5289,22.71,48,0,1.90,OM,1701899031
105,105,sesheke,-17.4759,24.2968,26.98,31,45,1.57,ZM,1701899035
160,160,ar rahad,12.7167,30.6500,22.69,18,68,4.33,SD,1701899043
195,195,fada,9.5667,8.3000,22.28,28,24,1.64,NG,1701899050


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df.copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
19,19,taoudenni,22.6783,-3.9836,22.55,11,100,2.87,ML,1701899022,
80,80,sur,22.5667,59.5289,22.71,48,0,1.90,OM,1701899031,
105,105,sesheke,-17.4759,24.2968,26.98,31,45,1.57,ZM,1701899035,
160,160,ar rahad,12.7167,30.6500,22.69,18,68,4.33,SD,1701899043,
195,195,fada,9.5667,8.3000,22.28,28,24,1.64,NG,1701899050,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
taoudenni - nearest hotel: No hotel found
sur - nearest hotel: Sur Hotel
sesheke - nearest hotel: Sisheke Lodge
ar rahad - nearest hotel: بيت عمر ٨٠٠ متر
fada - nearest hotel: No hotel found
guerrero negro - nearest hotel: Plaza sal paraiso
ndele - nearest hotel: No hotel found
chos malal - nearest hotel: Hospedaje Lemus
bardai - nearest hotel: No hotel found
alpine - nearest hotel: Ayres Lodge Alpine
sarmiento - nearest hotel: Ismar
salalah - nearest hotel: Muscat International Hotel
tsumeb - nearest hotel: Minen Hotel
oxkutzkab - nearest hotel: Hotel Puuc
ormara - nearest hotel: No hotel found
cooma - nearest hotel: Royal Hotel
khorixas - nearest hotel: iGowati Country Hotel and Lodge
west university place - nearest hotel: Residence Inn
wamba - nearest hotel: No hotel found
kaga bandoro - nearest hotel: No hotel found
gorom-gorom - nearest hotel: No hotel found
ak'ordat - nearest hotel: Omal Hotel
kidal - nearest hotel: No hotel found
barro alto - nearest hotel:

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
19,19,taoudenni,22.6783,-3.9836,22.55,11,100,2.87,ML,1701899022,No hotel found
80,80,sur,22.5667,59.5289,22.71,48,0,1.90,OM,1701899031,Sur Hotel
105,105,sesheke,-17.4759,24.2968,26.98,31,45,1.57,ZM,1701899035,Sisheke Lodge
160,160,ar rahad,12.7167,30.6500,22.69,18,68,4.33,SD,1701899043,بيت عمر ٨٠٠ متر
195,195,fada,9.5667,8.3000,22.28,28,24,1.64,NG,1701899050,No hotel found
228,228,guerrero negro,27.9769,-114.0611,24.93,30,0,4.45,MX,1701899024,Plaza sal paraiso
237,237,ndele,8.4109,20.6473,26.37,27,60,1.22,CF,1701899058,No hotel found
266,266,chos malal,-37.3781,-70.2709,24.13,23,62,4.03,AR,1701899064,Hospedaje Lemus
271,271,bardai,11.6500,7.2333,21.38,18,11,3.88,NG,1701898791,No hotel found
284,284,alpine,32.8350,-116.7664,24.01,35,0,3.09,US,1701898805,Ayres Lodge Alpine


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM", size = "Humidity", color ='red', 
                                   hover_cols=['Hotel Name', 'Country'])

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name,Country)